In [1]:
# # Run just once
# !pip install torchmetrics

In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import glob
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import warnings

import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, ConcatDataset
from torchvision import transforms, utils
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import F1

from Dataset import AgeDataset
from myutil import mapAttributes, save_aug, mapAge
from model import MyResNet18

In [3]:
warnings.filterwarnings(action='ignore') 

In [4]:
# # run just once! (takes about 30 sec.)
# base_path = "../input/data/train"
# df = pd.read_csv(os.path.join(base_path, "train.csv"))
# old_labels = df.loc[df.age>=60]
# myaug = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.ColorJitter(contrast=0.1),
#     transforms.RandomPosterize(bits=2),
#     transforms.ToTensor()
# ])
# save_aug(myaug, old_labels)

100%|██████████| 192/192 [00:24<00:00,  7.82it/s]


In [5]:
# set random seed (이렇게 하면 되는건가.. 흠)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
     
seed_everything(42)

## split train, val folders

In [6]:
base_path = "/opt/ml/input/data/train"

# combine two dataframes (original + augmented)
df1 = pd.read_csv(os.path.join(base_path, "train.csv"))
df2 = pd.read_csv(os.path.join(base_path, "old_path_augmented.csv"))
df = df1.append(df2)  # 2892 rows (= 2700 + 192)

y_data = df.apply(lambda x: mapAge(x['path']), axis=1)

x_train, x_val, y_train, y_val = train_test_split(df.index, y_data, test_size=0.2, random_state=42, stratify=y_data)
len(x_train), len(x_val)  # 2313, 579 확인!

(2313, 579)

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
EPOCHS = 30

## Dataset & Dataloader

In [8]:
df = pd.read_csv(os.path.join(base_path, "train.csv"))

train_dataset = AgeDataset(
    data_path = base_path,
    data = df.loc[x_train],
    transform = transforms.Compose([
         transforms.Resize((128, 128)),
         transforms.ToTensor()
    ])
)

val_dataset = AgeDataset(
    data_path = base_path,
    data = df.loc[x_val],
    transform = transforms.Compose([
         transforms.Resize((128, 128)),
         transforms.ToTensor()
    ])
)

100%|██████████| 579/579 [00:00<00:00, 20161.74it/s]


In [9]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=4,
    drop_last = True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=4,
    drop_last = True
)

# x, y = next(iter(train_loader))
# x.shape, y.shape  # (torch.Size([32, 3, 128, 128]), torch.Size([32]))

In [10]:
# model, loss, optm
resnet18 = MyResNet18(num_classes=3).to(DEVICE)
multi_criterion = nn.MultiLabelSoftMarginLoss(reduction='mean')
optm = torch.optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)

In [11]:
# calculates accuracy
def get_acc(y_pred, y_test):  # torch.Size([32, 3]) torch.Size([32])
    output = torch.argmax(y_pred, dim=1)
    correct = sum(output == y_test)/len(output)
    return torch.sum(correct)

# labels with proper dimension
def labeling(x, num_labels):
    output = torch.zeros((x.shape[0], num_labels))
    for i in range(x.shape[0]):
        output[i,x[i]] = 1
    return output

# F1 score
f1 = F1(num_classes=3)

## Train

In [12]:
# train

tr_writer = SummaryWriter("logs/exp_12/tr")
val_writer = SummaryWriter("logs/exp_12/val")

for ep in tqdm(range(EPOCHS)):

    # == training phase =================================
    train_loss_ep = 0
    train_accuracy_ep = 0
    train_f1_ep = 0
    for X, y in iter(train_loader):
        
        resnet18.train()
        
        # change dim for loss func
        _y = labeling(y.unsqueeze(1), 3)  # torch.Size([32, 3])
        X, _y = X.to(DEVICE), _y.to(DEVICE)
        
        predict = resnet18(X)  # torch.Size([32, 3])
        
        train_loss = multi_criterion(predict, _y)
        train_accuracy = get_acc(predict, y.squeeze().to(DEVICE))
        train_f1 = f1(torch.argmax(predict, 1).to("cpu"), y)
        
        optm.zero_grad()
    
        train_loss.backward()
        optm.step()
        
        train_loss_ep += train_loss
        train_accuracy_ep += train_accuracy
        train_f1_ep += train_f1
        # end of one dataloader
    
    train_loss_avg = train_loss_ep / len(train_loader)
    train_accuracy_avg = train_accuracy_ep / len(train_loader)
    train_f1_avg = train_f1_ep / len(train_loader)
    
    # tensorboard
    tr_writer.add_scalar(
        'train_loss_avg',
        train_loss_avg,
        ep
    )
    tr_writer.add_scalar(
        'train_accuracy_avg',
        train_accuracy_avg,
        ep
    )
    tr_writer.add_scalar(
        'train_f1_avg',
        train_f1_avg,
        ep
    )
    
    # === testing phase =================================
    with torch.no_grad():
        val_loss_ep = 0
        val_accuracy_ep = 0
        val_f1_ep = 0
        for X, y in iter(val_loader):
            ## testing phrase
            resnet18.eval()

            # change dim for loss func
            _y = labeling(y.unsqueeze(1), 3)

            X, _y = X.to(DEVICE), _y.to(DEVICE)
            predict = resnet18(X)

            val_loss = multi_criterion(predict, _y)
            val_accuracy = get_acc(predict, y.squeeze().to(DEVICE))
            val_f1 = f1(torch.argmax(predict, 1).to("cpu"), y)

            val_loss_ep += val_loss
            val_accuracy_ep += val_accuracy
            val_f1_ep += val_f1


        val_loss_avg = val_loss_ep / len(val_loader)
        val_accuracy_avg = val_accuracy_ep / len(val_loader)
        val_f1_avg = val_f1_ep / len(val_loader)
    
    # tensorboard
    val_writer.add_scalar(
        'val_loss_avg',
        val_loss_avg,
        ep
    )
    val_writer.add_scalar(
        'val_accuracy_avg',
        val_accuracy_avg,
        ep
    )
    val_writer.add_scalar(
        'val_f1_avg',
        val_f1_avg,
        ep
    )
    
    print("train_loss_avg:", train_loss_avg.item(), "train_accuracy_avg:", train_accuracy_avg.item(), "train_f1_avg:", train_f1_avg)
    print("val_loss_avg:", val_loss_avg.item(), "val_accuracy_avg:", val_accuracy_avg.item(), "val_f1_avg:", val_f1_avg)

  3%|▎         | 1/30 [00:24<11:42, 24.24s/it]

train_loss_avg: 0.3161337375640869 train_accuracy_avg: 0.810519814491272 train_f1_avg: tensor(0.8105)
val_loss_avg: 0.24205219745635986 val_accuracy_avg: 0.85317462682724 val_f1_avg: tensor(0.8532)


  7%|▋         | 2/30 [00:48<11:17, 24.19s/it]

train_loss_avg: 0.17072679102420807 train_accuracy_avg: 0.915099024772644 train_f1_avg: tensor(0.9151)
val_loss_avg: 0.22901587188243866 val_accuracy_avg: 0.8764881491661072 val_f1_avg: tensor(0.8765)


 10%|█         | 3/30 [01:12<10:52, 24.17s/it]

...

train_loss_avg: 0.007511787582188845 train_accuracy_avg: 0.9969678521156311 train_f1_avg: tensor(0.9970)
val_loss_avg: 0.39886611700057983 val_accuracy_avg: 0.8869048357009888 val_f1_avg: tensor(0.8869)


 97%|█████████▋| 29/30 [11:46<00:24, 24.28s/it]

train_loss_avg: 0.007497994229197502 train_accuracy_avg: 0.9961014986038208 train_f1_avg: tensor(0.9961)
val_loss_avg: 0.34848788380622864 val_accuracy_avg: 0.8990575671195984 val_f1_avg: tensor(0.8991)


100%|██████████| 30/30 [12:10<00:00, 24.35s/it]

train_loss_avg: 0.002167279366403818 train_accuracy_avg: 0.9994431138038635 train_f1_avg: tensor(0.9994)
val_loss_avg: 0.3560841977596283 val_accuracy_avg: 0.8960813879966736 val_f1_avg: tensor(0.8961)


In [13]:
# # How to Save

# best_val_loss = 0
# best_val_f1 = 0

# for ep in tqdm(range(EPOCHS)):
    
#     # train, valid ...
    
#     if best_val_f1 < val_f1_avg or best_val_loss >= val_loss_avg:
#         best_val_loss = val_loss_avg
#         best_val_f1 = val_f1_avg
#         torch.save(resnet18.state_dict(), f"{resnet18.__class__.__name__}_{ep}_loss{train_loss_avg:.2f}_acc{train_accuracy_avg:2f}.pt")
        
# #         torch.save({
# #             'epoch': ep,
# #             'model_state_dict': resnet18.state_dict(),
# #             'optimizer_state_dict': optm.state_dict(),
# #             'loss': train_loss_avg,
# #             }, f"saved/{resnet18.__class__.__name__}_{ep}_loss{train_loss_avg:.2f}_acc{train_accuracy_avg:2f}.pt")

